In [25]:
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
bucket = 'sxm-ecommerce-p66-location-data'
container = {'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',}
training_image = container[boto3.Session().region_name]
print(training_image)

arn:aws:iam::969580840594:role/service-role/AmazonSageMaker-ExecutionRole-20210707T102315
811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [27]:
data = pd.read_csv('data/road_or_other.csv')
#data = data.drop(['weird_imgs'], axis =1)
data.head()

,img,type,path
0,"21.331533, -157.865814_freq_1",0,sxm-ecommerce-p66-location-data/version2/train...
1,"21.331533, -157.865814_freq_2",0,sxm-ecommerce-p66-location-data/version2/train...
2,"21.331533, -157.865814_freq_3",1,sxm-ecommerce-p66-location-data/version2/train...
3,"21.331533, -157.865814_freq_4",0,sxm-ecommerce-p66-location-data/version2/train...
4,"28.782888, -97.045563_freq_1",0,sxm-ecommerce-p66-location-data/version2/train...


In [28]:
data['path2'] = [str(path).replace('sxm-ecommerce-p66-location-data/version2/training_freqs/', '') for path in data['path']]

# data['path'] = 'road' if data['type'] == 1 else 'other'
# str = "%s/%s" % ('road' if type == 1 else 'other', path)
# X.head(1)[0]

data.loc[data['type'] == 0, 'path'] = 'other/' + data['path2']
data.loc[data['type'] == 1, 'path'] = 'road/' + data['path2']

X = data.path
y = data.type

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True)

In [30]:
X_train.head()

378    other/img_45.504331, -122.475114_freq_3.png
277    other/img_39.392193, -119.790039_freq_2.png
136     road/img_35.484282, -120.665052_freq_1.png
234     road/img_38.664221, -121.272608_freq_3.png
297     other/img_39.875982, -75.121634_freq_2.png
Name: path, dtype: object

In [31]:
y_train.head()

378    0
277    0
136    1
234    1
297    0
Name: type, dtype: int64

In [32]:
result_train = pd.concat([y_train, X_train], axis=1, join="inner")


In [33]:
result_train.reset_index(inplace=True)

In [34]:
result_train

,index,type,path
0,378,0,"other/img_45.504331, -122.475114_freq_3.png"
1,277,0,"other/img_39.392193, -119.790039_freq_2.png"
2,136,1,"road/img_35.484282, -120.665052_freq_1.png"
3,234,1,"road/img_38.664221, -121.272608_freq_3.png"
4,297,0,"other/img_39.875982, -75.121634_freq_2.png"
...,...,...,...
315,288,0,"other/img_39.781769, -86.083413_freq_1.png"
316,44,1,"road/img_32.755714, -117.126091_freq_1.png"
317,275,0,"other/img_39.173099, -91.886520_freq_4.png"
318,61,0,"other/img_33.611694, -114.725608_freq_2.png"


In [35]:
result_train.to_csv('training.lst', header=None, index=None, sep='\t')

In [36]:
result_train

,index,type,path
0,378,0,"other/img_45.504331, -122.475114_freq_3.png"
1,277,0,"other/img_39.392193, -119.790039_freq_2.png"
2,136,1,"road/img_35.484282, -120.665052_freq_1.png"
3,234,1,"road/img_38.664221, -121.272608_freq_3.png"
4,297,0,"other/img_39.875982, -75.121634_freq_2.png"
...,...,...,...
315,288,0,"other/img_39.781769, -86.083413_freq_1.png"
316,44,1,"road/img_32.755714, -117.126091_freq_1.png"
317,275,0,"other/img_39.173099, -91.886520_freq_4.png"
318,61,0,"other/img_33.611694, -114.725608_freq_2.png"


In [37]:
result_test = pd.concat([y_test, X_test], axis=1, join="inner")
result_test.reset_index(inplace=True)
result_test.to_csv('validation.lst', header=None, index=None, sep='\t')

In [38]:
import pandas as pd
import numpy as np

testing = pd.read_csv('testing.lst', sep='\t', header=None, names = ['index', 'label', 'path'])
training = pd.read_csv('validation.lst', sep='\t', header=None, names = ['index', 'label', 'path'])


In [39]:
testing

,index,label,path
0,336,0,"other/img_41.260788, -96.177233_freq_1.png"
1,250,1,"road/img_38.798163, -90.559493_freq_3.png"
2,95,0,"other/img_34.493382, -118.199384_freq_4.png"
3,84,0,"other/img_34.221697, -118.467373_freq_1.png"
4,157,0,"other/img_36.461764, -94.302287_freq_2.png"
...,...,...,...
75,330,1,"road/img_40.826027, -91.208710_freq_3.png"
76,271,0,"other/img_39.014388, -94.736880_freq_4.png"
77,225,0,"other/img_38.542992, -90.192497_freq_2.png"
78,101,0,"other/img_34.865696, -111.764206_freq_2.png"


In [40]:
s3_resource = boto3.resource('s3')
# Move files into correct folders


# Copy object A as object B
bucket = 'sxm-ecommerce-p66-location-data'

for ind in testing.index:
    path = testing['path'][ind]
    label = testing['label'][ind]
    index = testing['index'][ind]
    oldPath = path.replace('other', 'version2/training_freqs')
    oldPath = oldPath.replace('road', 'version2/training_freqs')
    print(path)
    print(oldPath)
    print(index)
    
    copy_source = {
        'Bucket': bucket,
        'Key': oldPath
    }
    print("version2/validation/%s" % (path))
    response = s3_resource.meta.client.copy(copy_source, bucket, "version2/validation/%s" % (path))
    print(response)



other/img_41.260788, -96.177233_freq_1.png
version2/training_freqs/img_41.260788, -96.177233_freq_1.png
336
version2/validation/other/img_41.260788, -96.177233_freq_1.png
None
road/img_38.798163, -90.559493_freq_3.png
version2/training_freqs/img_38.798163, -90.559493_freq_3.png
250
version2/validation/road/img_38.798163, -90.559493_freq_3.png
None
other/img_34.493382, -118.199384_freq_4.png
version2/training_freqs/img_34.493382, -118.199384_freq_4.png
95
version2/validation/other/img_34.493382, -118.199384_freq_4.png
None
other/img_34.221697, -118.467373_freq_1.png
version2/training_freqs/img_34.221697, -118.467373_freq_1.png
84
version2/validation/other/img_34.221697, -118.467373_freq_1.png
None
other/img_36.461764, -94.302287_freq_2.png
version2/training_freqs/img_36.461764, -94.302287_freq_2.png
157
version2/validation/other/img_36.461764, -94.302287_freq_2.png
None
other/img_44.067844, -103.195831_freq_4.png
version2/training_freqs/img_44.067844, -103.195831_freq_4.png
367
version2

In [41]:
s3_resource = boto3.resource('s3')
# Move files into correct folders


# Copy object A as object B
bucket = 'sxm-ecommerce-p66-location-data'

for ind in training.index:
    path = training['path'][ind]
    label = training['label'][ind]
    index = training['index'][ind]
    oldPath = path.replace('other', 'version2/training_freqs')
    oldPath = oldPath.replace('road', 'version2/training_freqs')
    print(path)
    print(oldPath)
    print(index)
    
    copy_source = {
        'Bucket': bucket,
        'Key': oldPath
    }
    response = s3_resource.meta.client.copy(copy_source, bucket, "version2/training/%s" % (path))
    print(response)


other/img_45.504331, -122.475114_freq_3.png
version2/training_freqs/img_45.504331, -122.475114_freq_3.png
378
None
other/img_39.392193, -119.790039_freq_2.png
version2/training_freqs/img_39.392193, -119.790039_freq_2.png
277
None
road/img_35.484282, -120.665052_freq_1.png
version2/training_freqs/img_35.484282, -120.665052_freq_1.png
136
None
road/img_38.664221, -121.272608_freq_3.png
version2/training_freqs/img_38.664221, -121.272608_freq_3.png
234
None
other/img_39.875982, -75.121634_freq_2.png
version2/training_freqs/img_39.875982, -75.121634_freq_2.png
297
None
other/img_28.782888, -97.045563_freq_4.png
version2/training_freqs/img_28.782888, -97.045563_freq_4.png
7
None
road/img_33.037973, -111.378656_freq_3.png
version2/training_freqs/img_33.037973, -111.378656_freq_3.png
58
None
other/img_47.138551, -122.525624_freq_4.png
version2/training_freqs/img_47.138551, -122.525624_freq_4.png
395
None
road/img_46.316134, -117.988836_freq_2.png
version2/training_freqs/img_46.316134, -117.988